In [11]:
#cleaned_lines

In [ ]:
#pip install "PyPDF2" "chromadb==0.4.0" "fastapi<2.0.0" "langchain==0.2.0" "pydantic==1.10.19" "langchain-community" "sentence-transformers" 

In [1]:
from PyPDF2 import PdfReader
def read_pdf(file_path):
    # Initialize the reader for the PDF file
    
    reader = PdfReader(file_path)
    
    # Extract text from each page and store it in a variable
    pdf_text = ""
    for page in reader.pages:
        pdf_text += page.extract_text() + "\n"  # Adding a newline for separation between pages
  
    text = pdf_text.replace("\n", " ")
    
    return text
    #return page

# Reading PDF reports and remove sensitive data

In [1]:
from PyPDF2 import PdfReader
def read_pdf(file_path):
    # Initialize the reader for the PDF file
    
    reader = PdfReader(file_path)
    
    # Extract text from each page and store it in a variable
    pdf_text = ""
    for page in reader.pages:
        pdf_text += page.extract_text() + "\n"  # Adding a newline for separation between pages
  
    text = pdf_text.replace("\n", " ")
    
    return text
    #return page

In [2]:
import re
def remove_sensitive(text):
    text = re.sub(r"Patient\s+Name\s+:\s+.*?\s+(?=Patient\s+ID)", "", text)#removing patient name
    text = re.sub(r"Referring\s+Physician\s+.*?\s+Report", "Report", text)
    text = re.sub(r"Dr [A-Za-z\s]+MD\s*|Reg No:\s*\d+", "", text).strip()
    return text


# Creating field for json and removing other sensitive data

In [3]:
import re
import json
def text_json(extracted_data):
# Regex patterns to extract fields
    patterns = {
        "Patient ID": r"Patient\s+ID\s*:\s*(\S+)",
        "Date": r"Report\s+Date\s+/ Time\s+:\s+(\d{1,2}\s+\w+\s+\d{4})",
        "Time": r"(\d{2}:\d{2}:\d{2})",
        "Patient age": r"Patient\s+age\s+/\s+Sex\s+:\s+(\d{3}Y)",
        "Sex": r"Sex\s*:\s*\d{3}[A-Za-z]*\s*/\s*(\w)",
        "TECHNIQUE": r"TECHNIQUE\s*:\s*(.*?)\s*FINDINGS",
        "FINDINGS": r"FINDINGS\s*:\s*(.*?)\s*IMPRESSION",
        "IMPRESSION": r"IMPRESSION\s*:\s*(.*)"
    }
    
    # Extract fields using regex
    data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, extracted_data, re.DOTALL)
        if match:
            data[key] = match.group(1).strip()
    
    # removes the unwanted portion from the "FINDINGS" and "IMPRESSION" field while keeping the rest of the content intact.
    try:
        findings = data["FINDINGS"]
        impression = data["IMPRESSION"]
        updated_findings = re.sub(r"Patient\s+ID\s*:\s*(\S+).*Report\s+Date\s+/ Time\s+:\s+(\d{1,2}\s+\w+\s+\d{4})\s*(\S+)", "", findings).strip()
        updated_impression = re.sub(r"Patient\s+ID\s*:\s*(\S+).*Report\s+Date\s+/ Time\s+:\s+(\d{1,2}\s+\w+\s+\d{4})\s*(\S+)", "", impression).strip()

        # Update the Data
        data["FINDINGS"] = updated_findings
        data["IMPRESSION"] = updated_impression
    except:
        print("no unwanted found")
    # Update the JSON
    json_data = json.dumps(data, indent=4)
    
    return json_data
    

In [4]:
file_path = r"D:\LLM RAG\Data\MO.pdf"
print(text_json(remove_sensitive(read_pdf(file_path))))

{
    "Patient ID": "MR0000444444",
    "Date": "20 April  2020",
    "Time": "10:16:04",
    "Patient age": "075Y",
    "Sex": "M",
    "TECHNIQUE": "T2 FSE Axials  / Sagittals  & Coronals.   T1 &T2  FLAIR  Axials.  DWI,  GRE  Axials.    3D TOF  MR Angiography  of Intracranial  Arteries.",
    "FINDINGS": "Large wedge shaped lesion with restricted diffusion low ADC values and hyperintensities  in T2 & FLAIR images noted involving capsuloganglionic region, parietal lobe, adjacent  frontal insular cortex, temporal lobe in right side.   Focal FLAIR hyperintensities without restricted diffusion in bilateral posterior  periventricular wh ite matter, left corona radiata and centrum semiovale.   Curvilinear  blooming  SWI hypointensities  in distal  right  MCA.   Cerebellum, 4th ventricle, brain stem & CP angle regions are within normal limits.  Sella, suprasellar & parasellar areas are normal.   The ex tracerebral spaces and supratentorial ventricular system are normal.  Rest of the cerebra

# Processing multiple pdf file for chromadb format

In [5]:
# Process all PDF files in the folder
import os
folder_path = r"D:/LLM RAG/Data/"
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        print(pdf_path)
        extracted_data = read_pdf(pdf_path)
        remove_sensitive_data = remove_sensitive(extracted_data)
        extracted_json = text_json(remove_sensitive_data)
        ## Parse the JSON strings into dictionaries and append to the list
        all_data.append(json.loads(extracted_json))
#print(all_data)

D:/LLM RAG/Data/MO.pdf
D:/LLM RAG/Data/RA.pdf
D:/LLM RAG/Data/VA.pdf


In [6]:
all_data

[{'Patient ID': 'MR0000444444',
  'Date': '20 April  2020',
  'Time': '10:16:04',
  'Patient age': '075Y',
  'Sex': 'M',
  'TECHNIQUE': 'T2 FSE Axials  / Sagittals  & Coronals.   T1 &T2  FLAIR  Axials.  DWI,  GRE  Axials.    3D TOF  MR Angiography  of Intracranial  Arteries.',
  'FINDINGS': 'Large wedge shaped lesion with restricted diffusion low ADC values and hyperintensities  in T2 & FLAIR images noted involving capsuloganglionic region, parietal lobe, adjacent  frontal insular cortex, temporal lobe in right side.   Focal FLAIR hyperintensities without restricted diffusion in bilateral posterior  periventricular wh ite matter, left corona radiata and centrum semiovale.   Curvilinear  blooming  SWI hypointensities  in distal  right  MCA.   Cerebellum, 4th ventricle, brain stem & CP angle regions are within normal limits.  Sella, suprasellar & parasellar areas are normal.   The ex tracerebral spaces and supratentorial ventricular system are normal.  Rest of the cerebral parenchyma is 

# Converting JSON to ChromaDB format

In [9]:
from langchain.schema import Document

documents = [
    Document(page_content=f"{item['Patient ID']} {item['Patient age']} {item['Sex']} {item['TECHNIQUE']} {item['FINDINGS']} {item['IMPRESSION']}", metadata={})
    for item in all_data
]


In [10]:
# Verify the result
for doc in documents:
    print(f"Page Content: {doc.page_content}, Metadata: {doc.metadata}")

Page Content: MR0000444444 075Y M T2 FSE Axials  / Sagittals  & Coronals.   T1 &T2  FLAIR  Axials.  DWI,  GRE  Axials.    3D TOF  MR Angiography  of Intracranial  Arteries. Large wedge shaped lesion with restricted diffusion low ADC values and hyperintensities  in T2 & FLAIR images noted involving capsuloganglionic region, parietal lobe, adjacent  frontal insular cortex, temporal lobe in right side.   Focal FLAIR hyperintensities without restricted diffusion in bilateral posterior  periventricular wh ite matter, left corona radiata and centrum semiovale.   Curvilinear  blooming  SWI hypointensities  in distal  right  MCA.   Cerebellum, 4th ventricle, brain stem & CP angle regions are within normal limits.  Sella, suprasellar & parasellar areas are normal.   The ex tracerebral spaces and supratentorial ventricular system are normal.  Rest of the cerebral parenchyma is normal is normal.   Midline structures and corpus callosum are normal.  No haemorrhagic pathology.   No extraaxial  coll

# Creating chunks from document from chromaDB (vector database)
# Using all-MiniLM-L6-v2 embedings
# Saving the data into chromaDB

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# -------------------------------
# 1. Load JSON Data
# -------------------------------

#documents = loader.load()

# -------------------------------
# 2. Split Data into Chunks (Optional for Large Content)
# -------------------------------
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(documents)

# -------------------------------
# 3. Use Hugging Face Embeddings (all-MiniLM-L6-v2)
# -------------------------------
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# -------------------------------
# 4. Store Embeddings in ChromaDB
# -------------------------------
chroma_db4 = Chroma.from_documents(split_documents, embeddings, persist_directory="./chroma_sample_db")

# Save embeddings to the ChromaDB directory
chroma_db4.persist()
print("✅ Data has been successfully inserted into ChromaDB!")

C:\Users\bhowm\.conda\envs\bioinfo\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\bhowm\.conda\envs\bioinfo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Data has been successfully inserted into ChromaDB!


C:\Users\bhowm\.conda\envs\bioinfo\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

# Test Queries

In [14]:
# Test Queries
retriever = chroma_db4.as_retriever()
test_queries = ["Patient is non-cooperative"]

# Run Queries
for query in test_queries:
    results = retriever.get_relevant_documents(query,k=2)
    print(f"🔍 Query: {query}")
    
    for i, doc in enumerate(results[:2]):  # Limit to top 2 results
        print(f"💡 Result {i+1}: {doc.page_content}")
    print("-" * 50)


🔍 Query: Patient is non-cooperative
💡 Result 1: are normal.  Rest of the cerebral parenchyma is normal.   Midline  structures  and corpus  callosum  are normal.   No haemorrhagic pathology. No extraaxial collection seen.  MR Angiography shows : (Patient is not cooperative)   Distal right MCA branches are not well visualized.   Small  stenotic  segment  in distal  left MCA.   Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         Fetal  type of A1 segment  of right  ACA  -- Normal  varient.   Hypoplastic  right
💡 Result 2: MCA branches are not well visualized, possible due to slow flow  status / occlusion.   * Small  stenotic  segment  in distal  left MCA.   * Mild  stenosis  in cavernous  and infrapetrous  parts  of right  ICA.   * Hypoplastic  right  vertebral  artery.       Consultant Radiologist
--------------------------------------------------


In [13]:
pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 8.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure NLTK components are available
nltk.download('punkt')

# Define a reference document (ground truth)
reference_text = "Patient is not cooperative"
reference_tokens = [nltk.word_tokenize(reference_text.lower())]  # Tokenize & lowercase

# Test Query
test_query = "Patient is not cooperative"

# Retrieve relevant documents
results = retriever.get_relevant_documents(test_query, k=1)

# Compute BLEU Score for each retrieved document
for i, doc in enumerate(results[:1]):  # Top 2 results
    candidate_tokens = nltk.word_tokenize(doc.page_content.lower())  # Tokenize candidate
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, 
                               smoothing_function=SmoothingFunction().method1)

    print(f"💡 Result {i+1}: {doc.page_content}")
    print(f"🔹 BLEU Score: {bleu_score:.4f}")  # Print BLEU score
    print("-" * 50)


💡 Result 1: are normal.  Rest of the cerebral parenchyma is normal.   Midline  structures  and corpus  callosum  are normal.   No haemorrhagic pathology. No extraaxial collection seen.  MR Angiography shows : (Patient is not cooperative)   Distal right MCA branches are not well visualized.   Small  stenotic  segment  in distal  left MCA.   Mild  stenosis  seen in cavernous  and infrapetrous  parts  of right  ICA.         Fetal  type of A1 segment  of right  ACA  -- Normal  varient.   Hypoplastic  right
🔹 BLEU Score: 0.0278
--------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhowm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
